In [45]:
import pandas as pd

file_paths = {
    'international_trade': 'international_trade.csv',
    'test': 'test.csv',
    'train': 'train.csv',
    'sample_submission': 'sample_submission.csv'  # 여기에 콤마가 없으면 다음 줄에서 SyntaxError가 발생합니다.
}


# 각 파일을 데이터프레임으로 읽기
dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

# 각 데이터프레임의 처음 5줄 출력
for name, df in dataframes.items():
    print(f"{name} 데이터의 처음 5줄:")
    print(df.head(), "\n")


international_trade 데이터의 처음 5줄:
        기간                        품목명   수출 중량  수출 금액    수입 중량  수입 금액  무역수지
0  2019-01  토마토(신선한 것이나 냉장한 것으로 한정한다)  356571    990        0      0   990
1  2019-01                         양파  821330    222  4003206   1118  -896
2  2019-01                         쪽파      60      1    93405    128  -127
3  2019-01       꽃양배추와 브로콜리(broccoli)     160      1   638913    563  -562
4  2019-01                  방울다다기 양배추       0      0     7580     38   -38 

test 데이터의 처음 5줄:
                ID   timestamp item corporation location
0  TG_A_J_20230304  2023-03-04   TG           A        J
1  TG_A_J_20230305  2023-03-05   TG           A        J
2  TG_A_J_20230306  2023-03-06   TG           A        J
3  TG_A_J_20230307  2023-03-07   TG           A        J
4  TG_A_J_20230308  2023-03-08   TG           A        J 

train 데이터의 처음 5줄:
                ID   timestamp item corporation location  supply(kg)  \
0  TG_A_J_20190101  2019-01-01   TG           A        J         

In [2]:
dataframes['international_trade']['timestamp'] = pd.to_datetime(dataframes['international_trade']['기간'], format='%Y-%m')
train_timestamp_format = dataframes['train']['timestamp'].dtypes
test_timestamp_format = dataframes['test']['timestamp'].dtypes


In [3]:
unique_trade_items = dataframes['international_trade']['품목명'].unique()
unique_train_items = dataframes['train']['item'].unique()


In [5]:
monthly_trade_stats = dataframes['international_trade'].groupby(dataframes['international_trade']['timestamp'].dt.to_period('M')).agg({
    '수출 중량': 'sum',
    '수입 중량': 'sum',
    '무역수지': 'sum'
}).reset_index()

monthly_trade_stats['timestamp'] = monthly_trade_stats['timestamp'].dt.to_timestamp()


In [7]:
# 'train' 데이터프레임의 'timestamp' 컬럼을 datetime 타입으로 변환
dataframes['train']['timestamp'] = pd.to_datetime(dataframes['train']['timestamp'])

# 'test' 데이터프레임의 'timestamp' 컬럼을 datetime 타입으로 변환
dataframes['test']['timestamp'] = pd.to_datetime(dataframes['test']['timestamp'])

# 'monthly_trade_stats' 데이터프레임의 'timestamp' 컬럼을 datetime 타입으로 확인
monthly_trade_stats['timestamp'] = pd.to_datetime(monthly_trade_stats['timestamp'])

# 다시 병합을 시도합니다.
train_with_stats = dataframes['train'].merge(monthly_trade_stats, on='timestamp', how='left')
train_with_stats_filled = train_with_stats.fillna(method='ffill')
test_with_stats = dataframes['test'].merge(monthly_trade_stats, on='timestamp', how='left')
test_with_stats_filled = test_with_stats.fillna(method='ffill')


In [12]:
# 범주형 변수에 대해 원-핫 인코딩을 수행
categorical_columns = ['item', 'corporation', 'location']
train_encoded = pd.get_dummies(train_with_stats_filled, columns=categorical_columns)
test_encoded = pd.get_dummies(test_with_stats_filled, columns=categorical_columns)

# 'timestamp' 컬럼을 연도, 월, 일, 요일로 분해
train_encoded['year'] = train_encoded['timestamp'].dt.year
train_encoded['month'] = train_encoded['timestamp'].dt.month
train_encoded['day'] = train_encoded['timestamp'].dt.day
train_encoded['weekday'] = train_encoded['timestamp'].dt.weekday

# 원래의 'timestamp' 컬럼은 제거
train_encoded.drop('timestamp', axis=1, inplace=True)

# 테스트 데이터에 대해서도 동일한 처리를 수행
test_encoded['year'] = test_encoded['timestamp'].dt.year
test_encoded['month'] = test_encoded['timestamp'].dt.month
test_encoded['day'] = test_encoded['timestamp'].dt.day
test_encoded['weekday'] = test_encoded['timestamp'].dt.weekday

# 테스트 데이터에서도 'timestamp' 컬럼은 제거
test_encoded.drop('timestamp', axis=1, inplace=True)



In [22]:
from sklearn.model_selection import train_test_split

# train_encoded 데이터를 학습 세트와 검증 세트로 분할
X = train_encoded.drop(columns=['price(원/kg)', 'ID', 'supply(kg)'], errors='ignore')
y = train_encoded['price(원/kg)']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(
    n_estimators=10,  # 트리의 개수
    max_depth=3,      # 트리의 최대 깊이
    learning_rate=0.1,  # 학습률
    colsample_bytree=0.8,  # 트리를 구성할 때 특성 샘플링 비율
    subsample=0.8,  # 개별 트리를 훈련할 때 사용할 샘플 비율
    objective='reg:squarederror',  # 회귀 문제의 경우 'reg:squarederror' 사용
    random_state=42  # 결과의 재현성을 위한 랜덤 시드 설정
)


# 모델 학습
model.fit(X_train, y_train)

# 검증 세트에 대한 예측 수행
y_valid_pred = model.predict(X_valid)

# R^2 score 계산
r2_valid = r2_score(y_valid, y_valid_pred)

r2_valid


0.4545410911883372

In [23]:
y_valid_pred

array([1737.754  ,  595.689  ,  311.24188, ...,  373.00488,  364.01953,
        755.2709 ], dtype=float32)

In [26]:
# 모델 하이퍼파라미터를 증가시켜 재정의
model_improved = XGBRegressor(
    n_estimators=100,  # 트리의 개수를 증가
    max_depth=6,       # 트리의 깊이를 증가
    learning_rate=0.1,
    colsample_bytree=0.8,
    subsample=0.8,
    objective='reg:squarederror',
    random_state=42
)

# 모델 학습
model_improved.fit(X_train, y_train)

# 검증 세트에 대한 예측 수행
y_valid_pred = model_improved.predict(X_valid)

# R^2 score 계산
r2_valid = r2_score(y_valid, y_valid_pred)

r2_valid



0.7877340239557924

In [19]:
y_test_pred_improved

array([1113.9861 , -819.16364,  991.283  , ...,  424.7689 ,  387.96146,
        399.87335], dtype=float32)

In [38]:
# 모델 초기화
xgb_model = XGBRegressor(random_state=42)

# 제공된 모델 설정을 바탕으로 그리드 서치 파라미터 범위 설정
# 더 넓은 범위에서 성능이 좋을 가능성이 있는 값을 포함
param_grid = {
    'n_estimators': [80, 150],  # 주어진 모델보다 낮은 값을 포함하여 비교
    'max_depth': [5, 7],         # 주어진 모델의 값과 비슷하거나 더 높은 값을 포함
    'learning_rate': [0.05, 0.15],  # 학습률을 조금 더 높여 볼 수 있음
    'colsample_bytree': [0.6, 1.0],  # 모든 컬럼을 사용하는 경우 포함
    'subsample': [0.5, 1.0]         # 행의 샘플링 비율도 조정
}

# 그리드 서치 객체 초기화
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=3,                # 3-fold cross-validation
    scoring='r2',        # R^2 score를 기준으로 최적화
    verbose=1,           # 진행 상황에 대한 상세 출력
    n_jobs=-1            # 모든 CPU 코어 사용
)

# 학습 데이터 준비
X = train_encoded.drop(columns=['price(원/kg)', 'ID', 'supply(kg)'], errors='ignore')
y = train_encoded['price(원/kg)']

# 그리드 서치 실행
grid_search.fit(X, y)

# 결과 출력: 최적의 파라미터와 해당할 때의 R^2 스코어
best_parameters = grid_search.best_params_


best_parameters


Fitting 3 folds for each of 32 candidates, totalling 96 fits


{'colsample_bytree': 1.0,
 'learning_rate': 0.05,
 'max_depth': 5,
 'n_estimators': 80,
 'subsample': 0.5}

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 데이터 준비
# X와 y를 정의하는 코드는 이전에 실행한 코드와 동일하게 사용해야 합니다.
# X_train, X_valid, y_train, y_valid을 정의하는 코드 역시 이전에 실행한 코드를 그대로 사용해야 합니다.

# 최적의 하이퍼파라미터로 모델을 정의합니다 (best_parameters는 그리드 서치의 결과를 사용해야 함)
model = XGBRegressor(
    n_estimators=best_parameters['n_estimators'],
    max_depth=best_parameters['max_depth'],
    learning_rate=best_parameters['learning_rate'],
    colsample_bytree=best_parameters['colsample_bytree'],
    subsample=best_parameters['subsample'],
    objective='reg:squarederror',
    random_state=42
)

# 모델을 학습 세트에 학습시킵니다 (X_train과 y_train을 이전 코드에서 정의)
model.fit(X_train, y_train)

# 검증 세트에 대한 예측을 수행합니다 (X_valid는 이전 코드에서 정의)
y_valid_pred = model.predict(X_valid)

# R^2 스코어로 모델을 평가합니다 (y_valid는 이전 코드에서 정의)
r2 = r2_score(y_valid, y_valid_pred)

r2


0.745589674049661

In [47]:
import pandas as pd

# sample_submission.csv 파일을 로드합니다.
sample_submission = pd.read_csv('sample_submission.csv')

# test_encoded 데이터셋에서 'ID' 컬럼을 제외합니다.
X_test = test_encoded.drop(columns=['ID'])

# 테스트 데이터셋에 대한 예측을 수행합니다.
y_test_pred = model_improved.predict(X_test)

# 예측 결과를 sample_submission 데이터프레임의 'price(원/kg)' 컬럼에 저장합니다.
sample_submission['price(원/kg)'] = y_test_pred

# 완성된 제출 파일을 CSV 파일로 저장합니다.
submission_file_path = 'submission.csv'
sample_submission.to_csv(submission_file_path, index=False)
